In [1]:
import oracle
import numpy as np
import matplotlib.pyplot as plt

data = oracle.q1_get_cifar100_train_test(23746)
train_data = data[0]
test_data = data[1]

print("Test data : ", test_data[0])

Test data :  [[0.26274511218070984, 0.30980393290519714, 0.26274511218070984, 0.5215686559677124, 0.3921568691730499, 0.3490196168422699, 0.529411792755127, 0.5960784554481506, 0.45098039507865906, 0.21176470816135406, 0.29411765933036804, 0.2666666805744171, 0.3490196168422699, 0.32549020648002625, 0.33725491166114807, 0.37254902720451355, 0.40784314274787903, 0.3450980484485626, 0.1725490242242813, 0.2823529541492462, 0.2666666805744171, 0.26274511218070984, 0.3019607961177826, 0.3333333432674408, 0.29019609093666077, 0.32156863808631897, 0.29411765933036804], -1.0]


In [2]:
train_features = np.array([x[0] for x in train_data])
train_labels = np.array([x[1] for x in train_data])
test_features = np.array([x[0] for x in test_data])
test_labels = np.array([x[1] for x in test_data])   

print(train_features.shape)
print(train_labels.shape)
print(test_features.shape)
print(test_labels.shape)

(1000, 27)
(1000,)
(200, 27)
(200,)


In [18]:
def perceptron(train_features, train_labels, iterations):
    # Initialize weights
    w = np.zeros(train_features.shape[1])
    b = 0
    i = 0
    while i < iterations:
        #print(i)
        flag = 0
        for j in range(train_features.shape[0]):
            if train_labels[j] * (np.dot(w, train_features[j]) + b) <= 0:
                w += train_labels[j] * train_features[j]
                b += train_labels[j]
                flag = 1
        if flag == 0:
            break
        i += 1 
        

    #print("Iterations done: ", i)
    return w,i

In [19]:
# w, b, i = perceptron(train_features, train_labels, 100000000000)
# print("iterations: ", i)

    

In [20]:
def test_perceptron(iterations,test_features, test_labels):
    w, i = perceptron(train_features, train_labels, iterations)
    misclassified = 0
    for j in range(test_features.shape[0]):
        if test_labels[j] * (np.dot(w, test_features[j])) <= 0:
            misclassified += 1
    return misclassified/test_features.shape[0]

In [21]:
iterations = [i * 1000 for i in range(1, 5)]
error = []
for i in iterations:
    print(i)
    error.append(test_perceptron(i, test_features, test_labels))
    
# plt.plot(iterations, error)
# plt.xlabel('Iterations')
# plt.ylabel('Error')
# plt.title('Error vs Iterations')
# plt.show()


1000
2000
3000
4000


In [22]:
from cvxopt import matrix, solvers
def SVM_with_slack(train_features, train_labels, C):
    n = train_features.shape[0]
    m = train_features.shape[1]
    P = np.zeros((m+1+n, m+1+n))
    for i in range(m):
        P[i][i] = 1
    q = np.zeros(m+1+n)
    for i in range(m+1, m+1+n):
        q[i] = C
    h = np.zeros(2*n)
    for i in range(n):
        h[i] = -1
    G = np.zeros((2*n, m+1+n))
    for i in range(n):
        for j in range(m):
            G[i][j] = -train_labels[i] * train_features[i][j]
        G[i][m] = -train_labels[i]
        G[i][m+1+i] = -1
    for i in range(n):
        G[n+i][m+1+i] = -1
    P = matrix(P)
    q = matrix(q)
    G = matrix(G)
    h = matrix(h)
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h)
    w = np.array(sol['x'][:m])
    b = sol['x'][m]
    w = w.reshape((m,))
    return w, b

In [23]:
train_features_separable = []
train_labels_separable = []
C = 10
w,b = SVM_with_slack(train_features, train_labels, C)
for i in range(train_features.shape[0]):
    if train_labels[i] * (np.dot(w, train_features[i]) + b) > 0:
        train_features_separable.append(train_features[i])
        train_labels_separable.append(train_labels[i])
train_features_separable = np.array(train_features_separable)
train_labels_separable = np.array(train_labels_separable)
print(train_features_separable.shape)
print(train_labels_separable.shape) 

(692, 27)
(692,)


In [25]:
def test_svm(test_features, test_labels,c):
    w, b = SVM_with_slack(train_features_separable, train_labels_separable, c)
    misclassified = 0
    for j in range(test_features.shape[0]):
        if test_labels[j] * (np.dot(w, test_features[j]) + b) <= 0:
            misclassified += 1
    return misclassified/test_features.shape[0]

In [26]:
c = [i * 10 for i in range(11, 15)]
c = [i+0.5 for i in range(112,120)]
c = [120]
error = []
for i in c:
    error.append(test_svm(test_features, test_labels, i))
# print(error)
# plt.plot(c, error)
# plt.xlabel('C')
# plt.ylabel('Error')
# plt.title('Error vs C')
# plt.show()

In [27]:
# w , i = perceptron(train_features_separable, train_labels_separable, 100000000000)
# print("iterations: ", i)

In [15]:
import cvxopt
import cvxopt.solvers

def gaussian_kernel(x1, x2, gamma):
    return np.exp(-gamma * np.linalg.norm(x1 - x2) ** 2)

def compute_kernel_matrix(X, gamma):
    n_samples = X.shape[0]
    K = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            K[i, j] = gaussian_kernel(X[i], X[j], gamma)
    return K

def train_kernelized_svm(X, y, C=1.0, gamma=0.5):
    n_samples, n_features = X.shape
    K = compute_kernel_matrix(X, gamma)
    
    P = cvxopt.matrix(np.outer(y, y) * K)
    q = cvxopt.matrix(-np.ones(n_samples))
    A = cvxopt.matrix(y, (1, n_samples), 'd')
    b = cvxopt.matrix(0.0)
    G = cvxopt.matrix(np.vstack((-np.eye(n_samples), np.eye(n_samples))))
    h = cvxopt.matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C)))
    
    solution = cvxopt.solvers.qp(P, q, G, h, A, b)
    alphas = np.ravel(solution['x'])
    
    support_vectors = alphas > 1e-5
    sv_X = X[support_vectors]
    sv_y = y[support_vectors]
    sv_alphas = alphas[support_vectors]
    
    b = np.mean([y_k - np.sum(sv_alphas * sv_y * K[support_vectors, k]) 
                 for k, y_k in enumerate(sv_y)])
    
    return sv_X, sv_y, sv_alphas, b, gamma

def test_kernelized(train_features,train_labels, gamma, C):
    sv_X, sv_y, sv_alphas, b, gamma = train_kernelized_svm(train_features, train_labels, C, gamma)
    misclassified = 0
    for j in range(test_features.shape[0]):
        prediction = b
        for i in range(sv_X.shape[0]):
            prediction += sv_alphas[i] * sv_y[i] * gaussian_kernel(sv_X[i], test_features[j], gamma)
        if prediction * test_labels[j] <= 0:
            misclassified += 1
    return misclassified/test_features.shape[0]



In [16]:
gammas = [0.1, 0.5, 1, 2, 5]
C_values = [0.1, 0.5, 1, 2, 5]
error = []
for gamma in gammas:
    for C in C_values:
        error.append(test_kernelized(train_features_separable, train_labels_separable, gamma, C))
print(error)

[0.35, 0.345, 0.35, 0.32, 0.32, 0.33, 0.325, 0.325, 0.315, 0.32, 0.335, 0.32, 0.325, 0.32, 0.325, 0.335, 0.32, 0.3, 0.305, 0.305, 0.34, 0.31, 0.295, 0.3, 0.295]
